In [1]:
%cd ../airflow
%pwd

/home/welberthime/Development/fiap/cartolafc/airflow


'/home/welberthime/Development/fiap/cartolafc/airflow'

In [2]:
from include import RawExtractor

extractor = RawExtractor(
    base_url="https://api.github.com/repos/henriquepgomide/caRtola/contents/data",
    path="./data/raw",
)

extractor.extract_static_files("2015")
extractor.extract_dynamic_files("2015")

In [3]:
import pandas as pd

In [48]:
clubes_df = pd.read_csv("./data/raw/2015/times_ids/1.csv")
clubes_df = clubes_df[["id", "cod.2018"]].rename(
    columns={"cod.2018": "olderID", "id": "clubeID"}
).drop_duplicates()

partidas_df = pd.read_csv("./data/raw/2015/partidas_ids/1.csv")

partidas_df = partidas_df.merge(
    right=clubes_df,
    left_on="CasaID",
    right_on="olderID",
    suffixes=(None, "_mandante"),
)

partidas_df = partidas_df.merge(
    right=clubes_df,
    left_on="VisitanteID",
    right_on="olderID",
    suffixes=(None, "_visitante"),
)

partidas_df["temporada"] = 2015
partidas_df = partidas_df.rename(
    columns={
        "ID": "partidaID",
        "clubeID": "clubeMandanteID",
        "clubeID_visitante": "clubeVisitanteID",
        "PlacarCasa": "mandantePlacar",
        "PlacarVisitante": "visitantePlacar",
        "Rodada": "rodada",
        "Resultado": "resultado",
    }
)

partidas_df = partidas_df[[
    "partidaID",
    "clubeMandanteID",
    "clubeVisitanteID",
    "mandantePlacar",
    "visitantePlacar",
    "rodada",
    "resultado",
    "temporada",
]]

partidas_df

,partidaID,clubeMandanteID,clubeVisitanteID,mandantePlacar,visitantePlacar,rodada,resultado,temporada
0,190053,276,262,2.0,1.0,1,Casa,2015
1,190234,275,262,4.0,2.0,19,Casa,2015
2,190276,266,262,1.0,3.0,23,Visitante,2015
3,190136,267,262,1.0,0.0,9,Casa,2015
4,190378,284,262,2.0,0.0,33,Casa,2015
...,...,...,...,...,...,...,...,...
375,190208,282,276,3.0,1.0,16,Casa,2015
376,190229,316,276,0.0,2.0,18,Visitante,2015
377,190371,294,276,1.0,2.0,32,Visitante,2015
378,190432,290,276,NaN,NaN,38,NaN,2015


In [49]:
clubes_df = pd.read_csv("./data/raw/2015/times_ids/1.csv")
clubes_df = clubes_df[["id", "cod.2018"]].rename(
    columns={"cod.2018": "olderID", "id": "clubeID"}
).drop_duplicates()

scouts_df = pd.read_csv("./data/raw/2015/scouts_raw/1.csv")

scouts_df = scouts_df.merge(
    right=clubes_df,
    left_on="ClubeID",
    right_on="olderID",
)

scouts_df = scouts_df.rename(
    columns={
        'AtletaID': "atletaID",
        'Rodada': "rodada",
        'Pontos': "pontos",
        'PontosMedia': "pontosMedia",
        'Preco': "preco", 
        'PrecoVariacao': "precoVariacao",
    },
)

house_by_rounds = partidas_df[["partidaID", "rodada", "clubeMandanteID"]].rename(
    columns={"clubeMandanteID": "clubeID"}
)
visitor_by_rounds = partidas_df[["partidaID", "rodada", "clubeVisitanteID"]].rename(
    columns={"clubeVisitanteID": "clubeID"}
)
club_by_rounds = pd.concat([house_by_rounds, visitor_by_rounds])

scouts_df = scouts_df.merge(
    right=club_by_rounds,
    on=["rodada", "clubeID"],
)

scouts_df = scouts_df[[
    "clubeID",
    "partidaID",
    "atletaID",
    "pontos",
    "pontosMedia",
    "preco",
    "precoVariacao",
    "FS",
    "PE",
    "A",
    "FT",
    "FD",
    "FF",
    "G",
    "I",
    "PP",
    "RB",
    "FC",
    "GC",
    "CA",
    "CV",
    "SG",
    "DD",
    "DP",
    "GS",
]]

scouts_df

,clubeID,partidaID,atletaID,pontos,pontosMedia,preco,precoVariacao,FS,PE,A,...,PP,RB,FC,GC,CA,CV,SG,DD,DP,GS
0,262,190053,81121,-2.30,-2.30,0.79,-2.21,0,1,0,...,0,0,0,0,1,0,0,0,0,0
1,262,190053,81219,0.00,0.00,1.00,0.00,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,262,190053,88072,0.00,0.00,1.00,0.00,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,262,190053,89258,0.00,0.00,1.00,0.00,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,262,190053,91263,0.00,0.00,1.00,0.00,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29715,294,190415,63194,4.80,5.16,8.87,-0.76,51,68,1,...,0,36,31,0,1,0,9,0,0,0
29716,294,190415,68773,-2.50,6.06,10.27,-1.89,32,39,1,...,0,12,35,0,4,1,0,0,0,0
29717,294,190415,93153,3.19,4.30,11.20,-0.27,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29718,294,190415,50284,8.00,4.80,11.65,0.20,5,14,0,...,0,0,1,0,3,0,13,41,0,25


In [54]:
clubes_df = pd.read_csv("./data/raw/2015/times_ids/1.csv")
clubes_df = clubes_df[["id", "cod.2018"]].rename(
    columns={"cod.2018": "olderID", "id": "clubeID"}
).drop_duplicates()

atletas_df = pd.read_csv("./data/raw/2015/jogadores/1.csv")

atletas_df = atletas_df.merge(
    right=clubes_df,
    left_on="ClubeID",
    right_on="olderID",
)

atletas_df.drop_duplicates(inplace=True)
atletas_df.rename(
    columns={
        "ID": "atletaID",
        "Apelido": "apelido",
        "PosicaoID": "posicaoID",
    },
    inplace=True
)

atletas_df = atletas_df[[
    "atletaID",
    "clubeID",
    "posicaoID",
    "apelido",
]]

atletas_df

,atletaID,clubeID,posicaoID,apelido
0,85882,285,2,Diogo
1,62023,285,4,D'Alessandro
2,38219,285,4,Alex
3,69325,285,4,Aránguiz
4,78435,285,5,Vitinho
...,...,...,...,...
796,91741,215,4,Luiz Meneses
797,90320,215,3,Igor
798,90319,215,3,André Baumer
799,80335,215,1,Vitor


In [55]:
posicoes_df = pd.read_csv("./data/raw/2015/posicoes_ids/1.csv")
posicoes_df.rename(
    columns={
        "Cod": "posicaoID",
        "Position": "nome",
        "abbr": "abreviacao",
    },
    inplace=True,
)

posicoes_df = posicoes_df[[
    "posicaoID",
    "nome",
    "abreviacao",
]]

posicoes_df

,posicaoID,nome,abreviacao
0,1,Goleiro,gol
1,2,Lateral,lat
2,3,Zagueiro,zag
3,4,Meia,mei
4,5,Atacante,ata
5,6,Técnico,tec


In [56]:
clubes_df = pd.read_csv("./data/raw/2015/times_ids/1.csv")
clubes_df.rename(
    columns={
        "id": "clubeID",
        "nome.cbf": "nome",
    },
    inplace=True,
)

clubes_df = clubes_df[["clubeID", "nome", "abreviacao"]]
clubes_df.drop_duplicates(subset=["clubeID"], keep="last", inplace=True)

clubes_df

,clubeID,nome,abreviacao
0,327,América - MG,AME
1,200,America - RN,OUT
2,373,Atlético - GO,ATL
3,282,Atlético - MG,ATL
6,293,Athlético-PR,ATL
7,314,Avaí - SC,AVA
8,265,Bahia - BA,BAH
9,263,Botafogo - RJ,BOT
10,203,Brasiliense,BRA
13,354,Ceará - CE,CEA
